In [1]:
import random
import pandas as pd
import numpy as np
import Covid19_model as CM
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import os.path
import math
import csv
from pyomo.environ import *
import lhsmdu
import time

# Set parameter

In [2]:
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.3e6

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(1)
action_store = []
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne, num_sample = 0, 0, 0, 20000
done = False
index = [0]
start_time = 6
for t in range(T):
    pset = env.para_sampling(100)
    para = pset[index[0]]
    action_buffer = [] 
    reward_buffer = []
    
    if budget > 0 and t> start_time:     
        for j in range(num_sample):  
            b0 = budget
            s0 = score
            ac = [random.random(),random.random(),random.random(),random.random(),random.random()]
            ac = [ac * n_site]
            ac = np.reshape(np.array(ac), (n_site, 5)).T
            
            action_buffer.append(ac) 
            state_ex, r, _, _, c = env.transition(state, ac, para)  
            s0 += r
            b0 -= c
            for i in range(t, T):
                if b0 > 0:
                    act = np.ones((5, n_site)) 
                    
                else:
                    act = np.zeros((5, n_site))
                next_ex, reward_ex, _, _, c = env.transition(state_ex, act, para)  
                s0 += reward_ex
                b0 -= c
                state_ex = next_ex.copy()          
            reward_buffer.append(s0)
               
        max_reward = np.argmax(np.array(reward_buffer))
        action = action_buffer[max_reward]
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, pset, 100, 1)

    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    action_store.append(action)
    ni += new_i
    ne += new_e
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-229.27
 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-229.27
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2300000.00, reward:-554.35
 Time: 1 | S: 25495797.0, E: 143.0, A: 0.0, I: 47.3, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-554.35
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 

 Time: 11 | S: 25321217.2, E: 578.7, A: 1080.3, I: 2111.6, D: 4.3, R: 171007.9, Budget: 2118679.69, reward:-13714.09
 Time: 11 | S: 25321352.0, E: 551.0, A: 1080.3, I: 2111.6, D: 4.3, R: 170994.0, Budget: 2118679.69, reward:-13714.09
 Time: 11 | new_e:314.8, cum_e:3099.3, new_i:387.6, cum_i:3112.8
[[0.91372071 0.91372071 0.91372071 0.91372071 0.91372071 0.91372071
  0.91372071 0.91372071 0.91372071 0.91372071 0.91372071 0.91372071
  0.91372071 0.91372071 0.91372071 0.91372071]
 [0.90695639 0.90695639 0.90695639 0.90695639 0.90695639 0.90695639
  0.90695639 0.90695639 0.90695639 0.90695639 0.90695639 0.90695639
  0.90695639 0.90695639 0.90695639 0.90695639]
 [0.95943641 0.95943641 0.95943641 0.95943641 0.95943641 0.95943641
  0.95943641 0.95943641 0.95943641 0.95943641 0.95943641 0.95943641
  0.95943641 0.95943641 0.95943641 0.95943641]
 [0.88004351 0.88004351 0.88004351 0.88004351 0.88004351 0.88004351
  0.88004351 0.88004351 0.88004351 0.88004351 0.88004351 0.88004351
  0.88004351 0.8

 Time: 18 | S: 25087457.6, E: 440.9, A: 956.2, I: 2722.2, D: 10.8, R: 404412.3, Budget: 1935728.36, reward:-19957.89
 Time: 18 | S: 25087643.0, E: 425.0, A: 956.2, I: 2722.2, D: 10.8, R: 404411.0, Budget: 1935728.36, reward:-19957.89
 Time: 18 | new_e:226.2, cum_e:4449.6, new_i:300.5, cum_i:5179.3
[[0.90060964 0.90060964 0.90060964 0.90060964 0.90060964 0.90060964
  0.90060964 0.90060964 0.90060964 0.90060964 0.90060964 0.90060964
  0.90060964 0.90060964 0.90060964 0.90060964]
 [0.94311497 0.94311497 0.94311497 0.94311497 0.94311497 0.94311497
  0.94311497 0.94311497 0.94311497 0.94311497 0.94311497 0.94311497
  0.94311497 0.94311497 0.94311497 0.94311497]
 [0.95868555 0.95868555 0.95868555 0.95868555 0.95868555 0.95868555
  0.95868555 0.95868555 0.95868555 0.95868555 0.95868555 0.95868555
  0.95868555 0.95868555 0.95868555 0.95868555]
 [0.98600952 0.98600952 0.98600952 0.98600952 0.98600952 0.98600952
  0.98600952 0.98600952 0.98600952 0.98600952 0.98600952 0.98600952
  0.98600952 0.9

 Time: 25 | S: 24866557.6, E: 253.5, A: 1244.4, I: 3084.5, D: 19.2, R: 624840.8, Budget: 1810238.87, reward:-27140.62
 Time: 25 | S: 24866737.0, E: 258.0, A: 1244.4, I: 3084.5, D: 19.2, R: 624836.0, Budget: 1810238.87, reward:-27140.62
 Time: 25 | new_e:169.5, cum_e:5990.0, new_i:273.5, cum_i:7267.6
[[0.06207686 0.06207686 0.06207686 0.06207686 0.06207686 0.06207686
  0.06207686 0.06207686 0.06207686 0.06207686 0.06207686 0.06207686
  0.06207686 0.06207686 0.06207686 0.06207686]
 [0.94090364 0.94090364 0.94090364 0.94090364 0.94090364 0.94090364
  0.94090364 0.94090364 0.94090364 0.94090364 0.94090364 0.94090364
  0.94090364 0.94090364 0.94090364 0.94090364]
 [0.99484645 0.99484645 0.99484645 0.99484645 0.99484645 0.99484645
  0.99484645 0.99484645 0.99484645 0.99484645 0.99484645 0.99484645
  0.99484645 0.99484645 0.99484645 0.99484645]
 [0.82492722 0.82492722 0.82492722 0.82492722 0.82492722 0.82492722
  0.82492722 0.82492722 0.82492722 0.82492722 0.82492722 0.82492722
  0.82492722 0

 Time: 32 | S: 24742053.3, E: 403.1, A: 734.6, I: 2721.5, D: 25.0, R: 750062.5, Budget: 1634834.40, reward:-32347.65
 Time: 32 | S: 24742215.0, E: 414.0, A: 734.6, I: 2721.5, D: 25.0, R: 750082.0, Budget: 1634834.40, reward:-32347.65
 Time: 32 | new_e:197.2, cum_e:7097.9, new_i:253.4, cum_i:8978.9
[[0.68538276 0.68538276 0.68538276 0.68538276 0.68538276 0.68538276
  0.68538276 0.68538276 0.68538276 0.68538276 0.68538276 0.68538276
  0.68538276 0.68538276 0.68538276 0.68538276]
 [0.56807977 0.56807977 0.56807977 0.56807977 0.56807977 0.56807977
  0.56807977 0.56807977 0.56807977 0.56807977 0.56807977 0.56807977
  0.56807977 0.56807977 0.56807977 0.56807977]
 [0.96946024 0.96946024 0.96946024 0.96946024 0.96946024 0.96946024
  0.96946024 0.96946024 0.96946024 0.96946024 0.96946024 0.96946024
  0.96946024 0.96946024 0.96946024 0.96946024]
 [0.98944482 0.98944482 0.98944482 0.98944482 0.98944482 0.98944482
  0.98944482 0.98944482 0.98944482 0.98944482 0.98944482 0.98944482
  0.98944482 0.9

...

 Time: 60 | S: 24311324.6, E: 273.2, A: 470.3, I: 1879.0, D: 49.4, R: 1182003.5, Budget: 1043895.79, reward:-49587.06
 Time: 60 | S: 24311350.0, E: 269.0, A: 470.3, I: 1879.0, D: 49.4, R: 1182079.0, Budget: 1043895.79, reward:-49587.06
 Time: 60 | new_e:130.5, cum_e:10750.2, new_i:166.6, cum_i:14259.6
[[0.34238881 0.34238881 0.34238881 0.34238881 0.34238881 0.34238881
  0.34238881 0.34238881 0.34238881 0.34238881 0.34238881 0.34238881
  0.34238881 0.34238881 0.34238881 0.34238881]
 [0.75342981 0.75342981 0.75342981 0.75342981 0.75342981 0.75342981
  0.75342981 0.75342981 0.75342981 0.75342981 0.75342981 0.75342981
  0.75342981 0.75342981 0.75342981 0.75342981]
 [0.99980606 0.99980606 0.99980606 0.99980606 0.99980606 0.99980606
  0.99980606 0.99980606 0.99980606 0.99980606 0.99980606 0.99980606
  0.99980606 0.99980606 0.99980606 0.99980606]
 [0.95559146 0.95559146 0.95559146 0.95559146 0.95559146 0.95559146
  0.95559146 0.95559146 0.95559146 0.95559146 0.95559146 0.95559146
  0.95559146

 Time: 67 | S: 24303450.7, E: 173.6, A: 648.0, I: 1845.4, D: 54.0, R: 1189828.2, Budget: 964368.52, reward:-53552.59
 Time: 67 | S: 24303521.0, E: 166.0, A: 648.0, I: 1845.4, D: 54.0, R: 1189893.0, Budget: 964368.52, reward:-53552.59
 Time: 67 | new_e:97.0, cum_e:11591.7, new_i:152.7, cum_i:15388.5
[[0.05491752 0.05491752 0.05491752 0.05491752 0.05491752 0.05491752
  0.05491752 0.05491752 0.05491752 0.05491752 0.05491752 0.05491752
  0.05491752 0.05491752 0.05491752 0.05491752]
 [0.35696556 0.35696556 0.35696556 0.35696556 0.35696556 0.35696556
  0.35696556 0.35696556 0.35696556 0.35696556 0.35696556 0.35696556
  0.35696556 0.35696556 0.35696556 0.35696556]
 [0.97815331 0.97815331 0.97815331 0.97815331 0.97815331 0.97815331
  0.97815331 0.97815331 0.97815331 0.97815331 0.97815331 0.97815331
  0.97815331 0.97815331 0.97815331 0.97815331]
 [0.92487546 0.92487546 0.92487546 0.92487546 0.92487546 0.92487546
  0.92487546 0.92487546 0.92487546 0.92487546 0.92487546 0.92487546
  0.92487546 0.

 Time: 74 | S: 24168322.3, E: 46.0, A: 426.5, I: 1489.3, D: 58.8, R: 1325657.2, Budget: 732104.27, reward:-55187.23
 Time: 74 | S: 24168403.0, E: 40.0, A: 426.5, I: 1489.3, D: 58.8, R: 1325719.0, Budget: 732104.27, reward:-55187.23
 Time: 74 | new_e:25.6, cum_e:11916.6, new_i:67.7, cum_i:16091.1
[[0.62563705 0.62563705 0.62563705 0.62563705 0.62563705 0.62563705
  0.62563705 0.62563705 0.62563705 0.62563705 0.62563705 0.62563705
  0.62563705 0.62563705 0.62563705 0.62563705]
 [0.85684774 0.85684774 0.85684774 0.85684774 0.85684774 0.85684774
  0.85684774 0.85684774 0.85684774 0.85684774 0.85684774 0.85684774
  0.85684774 0.85684774 0.85684774 0.85684774]
 [0.98869201 0.98869201 0.98869201 0.98869201 0.98869201 0.98869201
  0.98869201 0.98869201 0.98869201 0.98869201 0.98869201 0.98869201
  0.98869201 0.98869201 0.98869201 0.98869201]
 [0.97783684 0.97783684 0.97783684 0.97783684 0.97783684 0.97783684
  0.97783684 0.97783684 0.97783684 0.97783684 0.97783684 0.97783684
  0.97783684 0.977

 Time: 81 | S: 23997052.2, E: 21.8, A: 244.0, I: 937.0, D: 60.9, R: 1497684.0, Budget: 535587.77, reward:-56062.02
 Time: 81 | S: 23997147.0, E: 16.0, A: 244.0, I: 937.0, D: 60.9, R: 1497754.0, Budget: 535587.77, reward:-56062.02
 Time: 81 | new_e:11.1, cum_e:12090.8, new_i:35.9, cum_i:16462.6
[[0.52730639 0.52730639 0.52730639 0.52730639 0.52730639 0.52730639
  0.52730639 0.52730639 0.52730639 0.52730639 0.52730639 0.52730639
  0.52730639 0.52730639 0.52730639 0.52730639]
 [0.90865606 0.90865606 0.90865606 0.90865606 0.90865606 0.90865606
  0.90865606 0.90865606 0.90865606 0.90865606 0.90865606 0.90865606
  0.90865606 0.90865606 0.90865606 0.90865606]
 [0.99284183 0.99284183 0.99284183 0.99284183 0.99284183 0.99284183
  0.99284183 0.99284183 0.99284183 0.99284183 0.99284183 0.99284183
  0.99284183 0.99284183 0.99284183 0.99284183]
 [0.99193331 0.99193331 0.99193331 0.99193331 0.99193331 0.99193331
  0.99193331 0.99193331 0.99193331 0.99193331 0.99193331 0.99193331
  0.99193331 0.99193

 Time: 88 | S: 23754236.8, E: 6.4, A: 106.3, I: 504.6, D: 61.9, R: 1741084.0, Budget: 276493.40, reward:-56293.20
 Time: 88 | S: 23754312.0, E: 15.0, A: 106.3, I: 504.6, D: 61.9, R: 1741152.0, Budget: 276493.40, reward:-56293.20
 Time: 88 | new_e:4.6, cum_e:12132.8, new_i:15.1, cum_i:16615.8
[[0.00840476 0.00840476 0.00840476 0.00840476 0.00840476 0.00840476
  0.00840476 0.00840476 0.00840476 0.00840476 0.00840476 0.00840476
  0.00840476 0.00840476 0.00840476 0.00840476]
 [0.92819335 0.92819335 0.92819335 0.92819335 0.92819335 0.92819335
  0.92819335 0.92819335 0.92819335 0.92819335 0.92819335 0.92819335
  0.92819335 0.92819335 0.92819335 0.92819335]
 [0.99741735 0.99741735 0.99741735 0.99741735 0.99741735 0.99741735
  0.99741735 0.99741735 0.99741735 0.99741735 0.99741735 0.99741735
  0.99741735 0.99741735 0.99741735 0.99741735]
 [0.96276956 0.96276956 0.96276956 0.96276956 0.96276956 0.96276956
  0.96276956 0.96276956 0.96276956 0.96276956 0.96276956 0.96276956
  0.96276956 0.9627695

In [6]:
result_state = pd.DataFrame(state_buffer)
result_state.to_csv(r'rollout_low_state_16.csv')
result_new = pd.DataFrame(new_buffer)
result_new.to_csv(r'rollout_low_new_16.csv')

In [10]:
import pickle
pickle.dump(action_store, open("rollout_low_action.pth", 'wb'))

In [8]:
(time.time() - tic) / ( T - start_time + 1)

211.9301882014555

In [9]:
12136.0 + 16628.7

28764.7